## Setup

In [1]:
from dask_yarn import YarnCluster
from dask.distributed import Client

/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import (
/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: parse_timedelta is deprecated and will be removed in a future release. Please use dask.utils.parse_timedelta instead.
  from distributed.utils import (


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
# Create a cluster where each worker has 4 vCPU core and 8 GiB of memory:
cluster = YarnCluster(environment="/home/hadoop/environment.tar.gz",
                      worker_vcores = 4,
                      worker_memory = "8GiB"
                      )

# Scale cluster out to 2 such workers:
cluster.scale(2)

# Connect to the cluster (before proceeding, you should wait for workers to be registered by the dask scheduler, as below):
client = Client(cluster)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://172.31.23.226:8786
distributed.scheduler - INFO -   dashboard at:                    :41423
distributed.scheduler - INFO - Receive client connection: Client-9551f10c-f906-11ed-8246-0a20a5309805
distributed.core - INFO - Starting established connection


In [4]:
import statsmodels
print(statsmodels.__version__)

0.13.1


In [5]:
import linearmodels
print(linearmodels.__version__)

4.31


In [6]:
import dask.dataframe as dd

/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


## Data

In [7]:
data = dd.read_parquet("s3://trade-final-project-bucket/dataset/trades_ARDL.parquet",
                     engine='pyarrow').persist()

/home/hadoop/miniconda/lib/python3.8/site-packages/charset_normalizer/api.py:105: UserWarning: Trying to detect encoding from a tiny portion of (19) byte(s).
  warn('Trying to detect encoding from a tiny portion of ({}) byte(s).'.format(length))


In [8]:
data.head()

distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.31.94:39951', name: dask.worker_1, memory: 0, processing: 26>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.31.94:39951
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.31.91:35647', name: dask.worker_0, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.31.91:35647
distributed.core - INFO - Starting established connection


,country,year_quarter,export_import,hs2,value_log,ex_log,gdp_log,value_log_L1,value_log_L2,value_log_L3,...,D_value_log_L3,D_ex_log_L1,D_ex_log_L2,D_ex_log_L3,D_gdp_log_L1,D_gdp_log_L2,D_gdp_log_L3,D_value_log,D_ex_log,D_gdp_log
0,103,1988_1,1,16,10.394059,1.800474,17.369277,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103,1988_2,1,16,10.236239,1.776693,17.378146,10.394059,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.157821,-0.023781,0.008869
2,103,1988_3,1,16,10.563078,1.691605,17.424784,10.236239,10.394059,NaN,...,NaN,-0.023781,NaN,NaN,0.008869,NaN,NaN,0.326839,-0.085088,0.046638
3,103,1988_4,1,16,10.580048,1.713851,17.475342,10.563078,10.236239,10.394059,...,NaN,-0.085088,-0.023781,NaN,0.046638,0.008869,NaN,0.016970,0.022246,0.050558
4,103,1989_1,1,16,10.023402,1.661634,17.485582,10.580048,10.563078,10.236239,...,-0.157821,0.022246,-0.085088,-0.023781,0.050558,0.046638,0.008869,-0.556646,-0.052216,0.010240


In [9]:
len(data)

743693

In [9]:
data_d = data.dropna()

In [10]:
hs2_lst = data_d["hs2"].unique().compute()

In [11]:
len(hs2_lst)

88

## Export Estimation

In [16]:
def fit_ardl(data, meta=None):
    
    try:
        mod = linearmodels.PanelOLS.from_formula(
            '''D_value_log ~ 1 + value_log_L1 + ex_log_L1 + gdp_log_L1 
            + D_value_log_L1 + D_value_log_L2 + D_value_log_L3
            + D_ex_log + D_ex_log_L1 + D_ex_log_L2 + D_ex_log_L3
            + D_gdp_log + D_gdp_log_L2 + D_gdp_log_L3 + D_gdp_log_L3
            + EntityEffects + TimeEffects''', data=data, check_rank=False
        )
        result = mod.fit()
        return result
    except:
        return "failed"
    


In [17]:
results_ex = dict()

for hs2 in hs2_lst:
    df = data_d[(data_d["hs2"]==hs2)&(data_d["export_import"]==1)].compute()
    df = df.set_index(['year_quarter','country'])
    model = fit_ardl(df)
    results_ex[hs2] = model 

In [19]:
for i,k in results_ex.items():
    print(i, k)

16                           PanelOLS Estimation Summary                           
Dep. Variable:            D_value_log   R-squared:                        0.3517
Estimator:                   PanelOLS   R-squared (Between):              0.2049
No. Observations:                1776   R-squared (Within):               0.1856
Date:                Tue, May 23 2023   R-squared (Overall):              0.1884
Time:                        01:16:43   Log-likelihood                   -1989.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      66.678
Entities:                         137   P-value                           0.0000
Avg Obs:                       12.964   Distribution:                 F(13,1598)
Min Obs:                       1.0000                                           
Max Obs:                       19.000   F-statistic (robust):             66.678
                         

In [24]:
param_ex_lst = []
param_idx = []

for i, k in results_ex.items():
    try:
        param = k.params
        param_ex_lst.append(param)
        param_idx.append(i)
    except:
        continue

In [25]:
param_ex = pd.DataFrame(param_ex_lst)

In [26]:
param_ex.index = param_idx

In [27]:
param_ex.head()

,Intercept,D_ex_log,D_ex_log_L1,D_ex_log_L2,D_ex_log_L3,D_gdp_log,D_gdp_log_L2,D_gdp_log_L3,D_value_log_L1,D_value_log_L2,D_value_log_L3,ex_log_L1,gdp_log_L1,value_log_L1
16,-0.345824,-0.449035,0.016395,-0.250400,0.221264,0.525874,-0.916763,-0.663147,-0.491961,-0.317254,-0.177556,-0.198898,0.149524,-0.215943
68,1.217466,-0.052629,-0.121879,-0.125020,0.003779,-0.453443,0.072625,-0.112220,-0.485626,-0.279823,-0.158403,-0.017547,0.077532,-0.202289
79,-0.560455,0.004174,0.080357,-0.020066,-0.006089,-0.904287,0.934366,-1.190207,-0.431619,-0.232771,-0.062336,0.003330,0.211258,-0.262851
97,-4.984060,0.066898,0.755643,0.839711,0.079055,0.022445,-0.560143,0.229366,-0.255615,-0.159234,-0.119741,-0.565247,0.666006,-0.630370
41,-0.146375,-1.066002,0.014895,-0.039638,-0.101978,-0.075533,-0.969628,-0.068135,-0.426577,-0.257484,-0.188472,-0.181873,0.174100,-0.287768


In [28]:
len(param_ex)

87

In [30]:
param_ex_final = param_ex[["Intercept"]]
param_ex_final["ex_long"] = param_ex["ex_log_L1"]/param_ex["value_log_L1"]
param_ex_final["ex_short_L1"] = param_ex["D_ex_log_L1"]/param_ex["D_value_log_L1"]
param_ex_final["ex_short_L2"] = param_ex["D_ex_log_L2"]/param_ex["D_value_log_L2"]
param_ex_final["ex_short_L3"] = param_ex["D_ex_log_L3"]/param_ex["D_value_log_L3"]

<ipython-input-30-1aec0b84c8ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  param_ex_final["ex_long"] = param_ex["ex_log_L1"]/param_ex["value_log_L1"]
<ipython-input-30-1aec0b84c8ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  param_ex_final["ex_short_L1"] = param_ex["D_ex_log_L1"]/param_ex["D_value_log_L1"]
<ipython-input-30-1aec0b84c8ae>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [76]:
hs2 = pd.read_csv("s3://trade-final-project-bucket/dataset/hs2codes.csv")
hs2 = hs2.dropna()
hs2["hs2"]=hs2["hs2"].astype("int").astype("str")
hs2 = hs2[["hs2","hs2_name"]]

In [77]:
hs2.head()

,hs2,hs2_name
0,1,Live animals.
1,2,Meat and edible meat offal.
2,3,Fish and crustaceans molluscs and other aquat...
3,4,Dairy produce; birds' eggs; natural honey; edi...
4,5,Products of animal origin not elsewhere speci...


In [33]:
param_ex_final = param_ex_final.reset_index().merge(hs2, left_on='index', right_on='hs2').sort_values('ex_long',ascending=False)

In [35]:
p_ex_lst = []

for i, k in results_ex.items():
    try:
        p = k.pvalues
        p_ex_lst.append(p)
    except:
        continue

In [36]:
p_ex = pd.DataFrame(p_ex_lst)

p_ex.index = param_idx

In [37]:
param_ex_final_v = param_ex_final.merge(p_ex[["Intercept", "ex_log_L1", "D_ex_log_L1", "D_ex_log_L2", "D_ex_log_L3"]].reset_index(), on="index")
param_ex_final_v = param_ex_final_v.rename(columns={"Intercept_x":"Intercept","Intercept_y":"p_Intercept","ex_log_L1":"p_ex_long", 
                                  "D_ex_log_L1":"p_ex_short_L1", "D_ex_log_L2":"p_ex_short_L2", 
                                  "D_ex_log_L3":"p_ex_short_L3"})
param_ex_final_v

,index,Intercept,ex_long,ex_short_L1,ex_short_L2,ex_short_L3,hs2,hs2_name,p_Intercept,p_ex_long,p_ex_short_L1,p_ex_short_L2,p_ex_short_L3
0,24,-6.465214,3.518233,-4.345530,-7.381734,6.940103,24,Tobacco and manufactured tobacco substitutes.,2.425066e-01,2.978372e-02,0.036266,0.145608,0.613705
1,57,-9.010117,3.417721,1.692001,4.211809,3.185623,57,Carpets and other textile floor coverings.,2.624629e-03,5.032776e-04,0.156506,0.108917,0.527012
2,65,-12.847359,2.577149,-0.955138,1.072554,4.556038,65,Headgear and parts thereof.,9.095450e-08,3.824701e-08,0.107709,0.115265,0.000041
3,78,-7.066022,1.919140,0.137276,0.638882,4.240465,78,Lead and articles thereof.,7.997645e-02,1.034237e-01,0.931031,0.617462,0.075900
4,80,-5.111207,1.894582,-0.383609,0.664398,-0.092365,80,Tin and articles thereof.,6.673633e-02,1.220566e-03,0.575450,0.443275,0.953839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,52,4.385815,-0.855178,0.736294,2.871264,-0.610121,52,Cotton.,1.075078e-02,7.664287e-02,0.217883,0.003311,0.733853
83,15,7.591170,-0.971455,-0.841193,1.153456,3.473584,15,Animal or vegetable fats and oils and their cl...,1.562779e-04,3.563430e-02,0.163332,0.168118,0.008379
84,67,5.417846,-1.241071,0.809985,0.371293,0.702124,67,Prepared feathers and down and articles made o...,8.462687e-02,1.385638e-01,0.308825,0.480382,0.497830
85,75,7.578966,-1.464778,1.826076,4.404831,2.589577,75,Nickel and articles thereof.,2.426773e-03,1.033530e-02,0.121068,0.103346,0.326598


In [49]:
count = data_d[(data_d["export_import"]==1)].groupby("hs2").size().compute()

param_ex_final_v = param_ex_final_v.merge(pd.DataFrame(count),on="hs2")

In [53]:
param_ex_final_v.rename(columns={param_ex_final_v.columns[-1]:"count"},inplace=True)
param_ex_final_v.head()

,index,Intercept,ex_long,ex_short_L1,ex_short_L2,ex_short_L3,hs2,hs2_name,p_Intercept,p_ex_long,p_ex_short_L1,p_ex_short_L2,p_ex_short_L3,count
0,24,-6.465214,3.518233,-4.345530,-7.381734,6.940103,24,Tobacco and manufactured tobacco substitutes.,2.425066e-01,2.978372e-02,0.036266,0.145608,0.613705,833
1,57,-9.010117,3.417721,1.692001,4.211809,3.185623,57,Carpets and other textile floor coverings.,2.624629e-03,5.032776e-04,0.156506,0.108917,0.527012,1226
2,65,-12.847359,2.577149,-0.955138,1.072554,4.556038,65,Headgear and parts thereof.,9.095450e-08,3.824701e-08,0.107709,0.115265,0.000041,2210
3,78,-7.066022,1.919140,0.137276,0.638882,4.240465,78,Lead and articles thereof.,7.997645e-02,1.034237e-01,0.931031,0.617462,0.075900,845
4,80,-5.111207,1.894582,-0.383609,0.664398,-0.092365,80,Tin and articles thereof.,6.673633e-02,1.220566e-03,0.575450,0.443275,0.953839,1415


In [54]:
param_ex_final_v.to_csv("ex_results.csv")

## Import Estimation 

In [60]:
results_im = dict()

for hs2 in hs2_lst:
    df = data_d[(data_d["hs2"]==hs2)&(data_d["export_import"]==2)].compute()
    df = df.set_index(['year_quarter','country'])
    model = fit_ardl(df)
    results_im[hs2] = model 

In [61]:
for i,k in results_im.items():
    print(i, k)

16                           PanelOLS Estimation Summary                           
Dep. Variable:            D_value_log   R-squared:                        0.3474
Estimator:                   PanelOLS   R-squared (Between):              0.2529
No. Observations:                2756   R-squared (Within):              -0.3939
Date:                Tue, May 23 2023   R-squared (Overall):             -0.3383
Time:                        01:37:32   Log-likelihood                   -2727.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      105.12
Entities:                         137   P-value                           0.0000
Avg Obs:                       20.117   Distribution:                 F(13,2567)
Min Obs:                       1.0000                                           
Max Obs:                       31.000   F-statistic (robust):             105.12
                         

In [68]:
param_im_lst = []
param_idx = []

for i, k in results_ex.items():
    try:
        param = k.params
        param_im_lst.append(param)
        param_idx.append(i)
    except:
        continue

In [69]:
param_im = pd.DataFrame(param_im_lst)

In [70]:
param_im.index = param_idx

In [71]:
param_im.head()

,Intercept,D_ex_log,D_ex_log_L1,D_ex_log_L2,D_ex_log_L3,D_gdp_log,D_gdp_log_L2,D_gdp_log_L3,D_value_log_L1,D_value_log_L2,D_value_log_L3,ex_log_L1,gdp_log_L1,value_log_L1
16,-0.345824,-0.449035,0.016395,-0.250400,0.221264,0.525874,-0.916763,-0.663147,-0.491961,-0.317254,-0.177556,-0.198898,0.149524,-0.215943
68,1.217466,-0.052629,-0.121879,-0.125020,0.003779,-0.453443,0.072625,-0.112220,-0.485626,-0.279823,-0.158403,-0.017547,0.077532,-0.202289
79,-0.560455,0.004174,0.080357,-0.020066,-0.006089,-0.904287,0.934366,-1.190207,-0.431619,-0.232771,-0.062336,0.003330,0.211258,-0.262851
97,-4.984060,0.066898,0.755643,0.839711,0.079055,0.022445,-0.560143,0.229366,-0.255615,-0.159234,-0.119741,-0.565247,0.666006,-0.630370
41,-0.146375,-1.066002,0.014895,-0.039638,-0.101978,-0.075533,-0.969628,-0.068135,-0.426577,-0.257484,-0.188472,-0.181873,0.174100,-0.287768


In [72]:
len(param_im)

87

In [73]:
param_im_final = param_im[["Intercept"]]
param_im_final["ex_long"] = param_im["ex_log_L1"]/param_im["value_log_L1"]
param_im_final["ex_short_L1"] = param_im["D_ex_log_L1"]/param_im["D_value_log_L1"]
param_im_final["ex_short_L2"] = param_im["D_ex_log_L2"]/param_im["D_value_log_L2"]
param_im_final["ex_short_L3"] = param_im["D_ex_log_L3"]/param_im["D_value_log_L3"]

<ipython-input-73-59d7cd8f2eaf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  param_im_final["ex_long"] = param_im["ex_log_L1"]/param_im["value_log_L1"]
<ipython-input-73-59d7cd8f2eaf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  param_im_final["ex_short_L1"] = param_im["D_ex_log_L1"]/param_im["D_value_log_L1"]
<ipython-input-73-59d7cd8f2eaf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [78]:
param_im_final = param_im_final.reset_index().merge(hs2, left_on='index', right_on='hs2').sort_values('ex_long',ascending=False)

In [81]:
p_im_lst = []
p_idx = []

for i, k in results_im.items():
    try:
        p = k.pvalues
        p_im_lst.append(p)
        p_idx.append(i)
    except:
        continue

In [82]:
p_im = pd.DataFrame(p_im_lst)

p_im.index = p_idx

In [83]:
param_im_final_v = param_im_final.merge(p_im[["Intercept", "ex_log_L1", "D_ex_log_L1", "D_ex_log_L2", "D_ex_log_L3"]].reset_index(), on="index")
param_im_final_v = param_im_final_v.rename(columns={"Intercept_x":"Intercept","Intercept_y":"p_Intercept","ex_log_L1":"p_ex_long", 
                                  "D_ex_log_L1":"p_ex_short_L1", "D_ex_log_L2":"p_ex_short_L2", 
                                  "D_ex_log_L3":"p_ex_short_L3"})
param_im_final_v

,index,Intercept,ex_long,ex_short_L1,ex_short_L2,ex_short_L3,hs2,hs2_name,p_Intercept,p_ex_long,p_ex_short_L1,p_ex_short_L2,p_ex_short_L3
0,24,-6.465214,3.518233,-4.345530,-7.381734,6.940103,24,Tobacco and manufactured tobacco substitutes.,0.368325,0.708721,0.125644,0.212966,0.724959
1,57,-9.010117,3.417721,1.692001,4.211809,3.185623,57,Carpets and other textile floor coverings.,0.019631,0.719293,0.414848,0.449200,0.031068
2,65,-12.847359,2.577149,-0.955138,1.072554,4.556038,65,Headgear and parts thereof.,0.843535,0.020019,0.675340,0.827655,0.723703
3,78,-7.066022,1.919140,0.137276,0.638882,4.240465,78,Lead and articles thereof.,0.106212,0.151717,0.605512,0.664239,0.724238
4,80,-5.111207,1.894582,-0.383609,0.664398,-0.092365,80,Tin and articles thereof.,0.632359,0.146635,0.056142,0.958707,0.586219
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,52,4.385815,-0.855178,0.736294,2.871264,-0.610121,52,Cotton.,0.860359,0.806452,0.271699,0.337837,0.303190
83,15,7.591170,-0.971455,-0.841193,1.153456,3.473584,15,Animal or vegetable fats and oils and their cl...,0.946351,0.565349,0.031772,0.634917,0.881241
84,67,5.417846,-1.241071,0.809985,0.371293,0.702124,67,Prepared feathers and down and articles made o...,0.050227,0.363264,0.366295,0.621638,0.268173
85,75,7.578966,-1.464778,1.826076,4.404831,2.589577,75,Nickel and articles thereof.,0.390194,0.493630,0.539409,0.631336,0.140022


In [84]:
count = data_d[(data_d["export_import"]==2)].groupby("hs2").size().compute()

param_im_final_v = param_im_final_v.merge(pd.DataFrame(count),on="hs2")

In [85]:
param_im_final_v.rename(columns={param_im_final_v.columns[-1]:"count"},inplace=True)
param_im_final_v.head()

,index,Intercept,ex_long,ex_short_L1,ex_short_L2,ex_short_L3,hs2,hs2_name,p_Intercept,p_ex_long,p_ex_short_L1,p_ex_short_L2,p_ex_short_L3,count
0,24,-6.465214,3.518233,-4.345530,-7.381734,6.940103,24,Tobacco and manufactured tobacco substitutes.,0.368325,0.708721,0.125644,0.212966,0.724959,1940
1,57,-9.010117,3.417721,1.692001,4.211809,3.185623,57,Carpets and other textile floor coverings.,0.019631,0.719293,0.414848,0.449200,0.031068,2393
2,65,-12.847359,2.577149,-0.955138,1.072554,4.556038,65,Headgear and parts thereof.,0.843535,0.020019,0.675340,0.827655,0.723703,2520
3,78,-7.066022,1.919140,0.137276,0.638882,4.240465,78,Lead and articles thereof.,0.106212,0.151717,0.605512,0.664239,0.724238,1069
4,80,-5.111207,1.894582,-0.383609,0.664398,-0.092365,80,Tin and articles thereof.,0.632359,0.146635,0.056142,0.958707,0.586219,2226


In [86]:
param_im_final_v.to_csv("im_results.csv")